# Advanced Retinal Vessel Model 2
This notebook contains the implementation of an advanced retinal vessel segmentation model using multi-scale features, transformer blocks, and attention.

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import *

def multi_scale_features2(x, filters):
    scales = []
    for i, rate in enumerate([1, 2, 4, 8]):
        scaled = Conv2D(filters, kernel_size=3, dilation_rate=rate, padding='same', activation='relu')(x)
        scales.append(scaled)
    return Concatenate()(scales)

def transformer_block2(x, num_heads=4, ff_dim=256):
    projected_x = Dense(ff_dim)(x)
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim // num_heads)(projected_x, projected_x)
    add_skip = Add()([attention, projected_x])
    attention_output = LayerNormalization()(add_skip)
    ff_output = Dense(ff_dim, activation="relu")(attention_output)
    ff_output = Dense(projected_x.shape[-1])(ff_output)
    add_skip_ff = Add()([ff_output, attention_output])
    return LayerNormalization()(add_skip_ff)

def aspp_transformer_block2(x):
    aspp = multi_scale_features2(x, 64)
    transformer = transformer_block2(aspp, num_heads=4, ff_dim=x.shape[-1])
    return transformer

def attention_transformer_block2(x, g, filters):
    g1 = Conv2D(filters, kernel_size=1, activation='relu')(g)
    x1 = Conv2D(filters, kernel_size=1, activation='relu')(x)
    psi = Add()([g1, x1])
    psi = Conv2D(1, kernel_size=1, activation='sigmoid')(psi)
    attention_applied = Multiply()([x, psi])
    return transformer_block2(attention_applied, num_heads=4, ff_dim=filters)

def advanced_retinal_vessel_model2(input_shape):
    filters = [32, 64, 128, 256, 512]
    inputs = Input(input_shape)
    enc1 = multi_scale_features2(inputs, filters[0])
    pool1 = MaxPooling2D(pool_size=(2, 2))(enc1)
    enc2 = multi_scale_features2(pool1, filters[1])
    pool2 = MaxPooling2D(pool_size=(2, 2))(enc2)
    enc3 = multi_scale_features2(pool2, filters[2])
    pool3 = MaxPooling2D(pool_size=(2, 2))(enc3)
    enc4 = multi_scale_features2(pool3, filters[3])
    pool4 = MaxPooling2D(pool_size=(2, 2))(enc4)
    bottleneck = aspp_transformer_block2(pool4)
    up4 = UpSampling2D(size=(2, 2))(bottleneck)
    att4 = attention_transformer_block2(enc4, up4, filters[3])
    dec4 = Conv2D(filters[3], kernel_size=(3, 3), padding='same', activation='relu')(Concatenate()([up4, att4]))
    up3 = UpSampling2D(size=(2, 2))(dec4)
    att3 = attention_transformer_block2(enc3, up3, filters[2])
    dec3 = Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu')(Concatenate()([up3, att3]))
    up2 = UpSampling2D(size=(2, 2))(dec3)
    att2 = attention_transformer_block2(enc2, up2, filters[1])
    dec2 = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu')(Concatenate()([up2, att2]))
    up1 = UpSampling2D(size=(2, 2))(dec2)
    att1 = attention_transformer_block2(enc1, up1, filters[0])
    dec1 = Conv2D(filters[0], kernel_size=(3, 3), padding='same', activation='relu')(Concatenate()([up1, att1]))
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(dec1)
    model = models.Model(inputs, outputs, name="Advanced_Retinal_Vessel_Model2")
    return model